In [30]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp

In [31]:
#数据加载
data=pd.read_csv('F:/BI/第十二周/L22/santa/family_data.csv',index_col='family_id')#把family_id作为表格的索引
data

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...
4995,16,1,66,33,18,70,56,46,86,60,4
4996,88,66,20,17,26,54,81,91,59,48,2
4997,32,66,54,17,27,21,74,81,3,7,6


In [32]:
#n代表家庭成员的个数，如果满足第chioce需求，需要的penalty
def get_penalty(n,choice):
    penalty=0
    if choice==0:
        penalty=0
    if choice==1:
        penalty=50
    if choice==2:
        penalty=50+9*n
    if choice==3:
        penalty=100+9*n
    if choice==4:
        penalty=200+9*n
    if choice==5:
        penalty=200+18*n
    if choice==6:
        penalty=300+18*n
    if choice==7:
        penalty=300+36*n
    if choice==8:
        penalty=400+36*n
    if choice==9:
        penalty=500+(36+199)*n
    if choice>9:
        penalty=500+(36+398)*n
    return penalty

In [33]:
N_DAYS=100 #安排的天数
N_FAMILY=5000 #家庭的ID个数
MIN_OCCUPANCY=125#最小承载量
MAX_OCCUPANCY=300#最大承载量

pcost_mat=np.full(shape=(N_FAMILY,100),fill_value=999999)
for f in range(N_FAMILY):
    #家庭成员
    f_num=data.loc[f,'n_people']
    #print(f_num)
    #对于第f个家庭，初始化pcost-other choice下的penalty
    pcost_mat[f,:]=get_penalty(f_num,10)
    #计算choice0-9的penalty
    for choice in range(10):
        temp=data.loc[f][choice]#chioce的天数
        penalty=get_penalty(f_num,choice)
        pcost_mat[f,temp-1]=penalty
pcost_mat        

array([[2236, 2236, 2236, ..., 2236, 2236, 2236],
       [2236, 2236, 2236, ..., 2236, 2236, 2236],
       [1802, 1802, 1802, ..., 1802, 1802,    0],
       ...,
       [3104, 3104,  616, ..., 3104, 3104, 3104],
       [ 390, 2670, 2670, ..., 2670, 2670, 2670],
       [2236, 2236, 2236, ..., 2236, 2236, 2236]])

In [34]:
pcost_mat.shape

(5000, 100)

In [35]:
#计算accounting penalty矩阵
acost_mat=np.zeros(shape=(500,500),dtype=np.float64)
for i in range(acost_mat.shape[0]):
    for j in range(acost_mat.shape[1]):
        diff=abs(i-j)
        acost_mat[i,j]=max(0,(i-125)/400* i **(0.5+diff/50.0))
acost_mat        

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.31354104e+28, 1.16015634e+28, 1.02468267e+28, ...,
        2.07329520e+01, 2.34740633e+01, 2.65775778e+01],
       [1.52288730e+28, 1.34500277e+28, 1.18789647e+28, ...,
        2.35617801e+01, 2.08095894e+01, 2.35617801e+01],
       [1.76565289e+28, 1.55934897e+28, 1.37715019e+28, ...,
        2.67784790e+01, 2.36496052e+01, 2.08863179e+01]])

In [36]:
# 查看数据形状
acost_mat.shape

(500, 500)

In [37]:
FAMILY_SIZE=data['n_people'].values
FAMILY_SIZE

array([4, 4, 3, ..., 6, 5, 4], dtype=int64)

In [38]:
#DESIERD代表每个家庭的choice-1
DESIERD=data.values[:,:-1]-1
DESIERD

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]], dtype=int64)

In [39]:
# 先使用LP 对绝大部分家庭进行规划
# 再使用MIP 对剩余家庭进行规划

In [40]:
%%time
#线性规划 ,只以preference_cost为目标
def solveLP():
    #线性规划 优化器
    solver=pywraplp.Solver('AssignmentProblem',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    #solver=pywraplp.Solver('AssignmentProblem',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    x={} #family_id 在第j天是否参观
    #每一天有哪些家庭
    candidates=[[] for x in range(N_DAYS)]    
    for i in range(N_FAMILY):
        for j in DESIERD[i,:]:        
            candidates[j].append(i)
            x[i,j]=solver.BoolVar('x[%i,%i]'%(i,j))
    #每天参观的人数100个数，x[i,j] -0或1
    daily_occupancy=[solver.Sum([x[i,j]*FAMILY_SIZE[i] for i in candidates[j]]) for j in range(N_DAYS)]
    
    #每个家庭在10个choice中出现的情况
    family_presense=[solver.Sum(x[i,j] for j in DESIERD[i,:]) for i in range(N_FAMILY)]
    
    #定义目标函数 preference cost部分
    preference_cost=solver.Sum([pcost_mat[i,j] * x[i,j] for i in range(N_FAMILY) for j in DESIERD[i,:]] )
    
    #满足preference_cost最小
    solver.Minimize(preference_cost)
    
    #认为增加约束条件Constaints
    for j in range(N_DAYS-1):
        #当前天数不超过前一天+25
        solver.Add(daily_occupancy[j]-daily_occupancy[j+1]<=25)
        #solver.Add(daily_occupancy[j+1]-daily_occupancy[j]<=25)
        
        #每个家庭都在10个chioce中出现1次
    for i in range(N_FAMILY):
        solver.Add(family_presense[i]==1)
        #每天访问人数约束
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j]>=MIN_OCCUPANCY)
        solver.Add(daily_occupancy[j]<=MAX_OCCUPANCY)
    result=solver.Solve()
    
    temp=[(i,j,x[i,j].solution_value()) for i in range(N_FAMILY) for j in DESIERD[i,:] if x[i,j].solution_value()>0]
    print(solver.Objective().Value())
    #得到参观日期的安排
    df=pd.DataFrame(temp,columns=['family_id','day','result'])
    return df
result=solveLP() 

65827.15476190476
Wall time: 13.3 s


In [41]:
result

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5074,4995,15,1.0
5075,4996,87,1.0
5076,4997,31,1.0
5077,4998,91,1.0


In [42]:
#设置阈值
THRS=0.999 
assigned_df=result[result.result>THRS]#已经安排上的结果
assigned_df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5074,4995,15,1.0
5075,4996,87,1.0
5076,4997,31,1.0
5077,4998,91,1.0


In [43]:
#没有安排的，不为0和1
unassigned_df=result[(result.result<THRS) & ( result.result<1-THRS ) ]
unassigned_df

,family_id,day,result
191,188,19,3.768222e-15
265,262,31,5.024296e-15
275,270,40,9.943919e-14
624,612,9,2.983176e-14
1213,1192,6,7.850462e-16
1582,1557,20,2.014199e-15
1756,1728,17,5.338314e-15
1799,1770,44,1.232209e-13
2043,2011,24,1.507289e-15
3224,3181,13,1.608682e-14


In [45]:
#计算每天访问的人数（根据assigned_df）
assigned_df['family_size']=FAMILY_SIZE[assigned_df.family_id]
assigned_df
assigned_df

,family_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5074,4995,15,1.0,4
5075,4996,87,1.0,2
5076,4997,31,1.0,6
5077,4998,91,1.0,5


In [46]:
#按照day进行聚合
occupancy=assigned_df.groupby('day').family_size.sum().values
occupancy

array([295, 271, 297, 297, 269, 248, 222, 198, 196, 297, 296, 293, 268,
       246, 220, 239, 296, 285, 269, 244, 219, 186, 208, 299, 300, 296,
       271, 238, 212, 195, 295, 275, 250, 221, 195, 172, 148, 293, 271,
       247, 221, 196, 175, 149, 297, 272, 244, 222, 196, 173, 190, 281,
       256, 236, 206, 180, 159, 174, 233, 210, 186, 163, 140, 125, 115,
       223, 198, 165, 140, 125, 125, 122, 217, 195, 173, 147, 124, 122,
       119, 218, 200, 173, 138, 120, 125, 122, 246, 225, 192, 173, 150,
       125, 120, 223, 199, 173, 150, 125, 125, 125], dtype=int64)

In [47]:
min_occupancy=np.array([max(0,MIN_OCCUPANCY-x) for x in occupancy])
min_occupancy

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,
        0,  0,  0,  3,  0,  0,  0,  0,  1,  3,  6,  0,  0,  0,  0,  5,  0,
        3,  0,  0,  0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0],
      dtype=int64)

In [48]:
max_occupancy=np.array([MAX_OCCUPANCY -x for x in occupancy])
max_occupancy

array([  5,  29,   3,   3,  31,  52,  78, 102, 104,   3,   4,   7,  32,
        54,  80,  61,   4,  15,  31,  56,  81, 114,  92,   1,   0,   4,
        29,  62,  88, 105,   5,  25,  50,  79, 105, 128, 152,   7,  29,
        53,  79, 104, 125, 151,   3,  28,  56,  78, 104, 127, 110,  19,
        44,  64,  94, 120, 141, 126,  67,  90, 114, 137, 160, 175, 185,
        77, 102, 135, 160, 175, 175, 178,  83, 105, 127, 153, 176, 178,
       181,  82, 100, 127, 162, 180, 175, 178,  54,  75, 108, 127, 150,
       175, 180,  77, 101, 127, 150, 175, 175, 175], dtype=int64)

In [49]:
#使用整数规划进行求解
def solveIP(families,min_occupancy,max_occupancy):
    #创建求解器
    solver=pywraplp.Solver('AssignmentProblem',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    #需要安排的家庭
    n_families=len(families)
    
    x={} #family_id 在第j天是否参观
    #每一天有哪些家庭
    candidates=[[] for x in range(N_DAYS)] 
    
    for i in families:
        for j in DESIERD[i,:]:        
            candidates[j].append(i)
            x[i,j]=solver.BoolVar('x[%i,%i]'%(i,j))
    
    #每天参观的人数100个数，x[i,j] -0或1
    daily_occupancy=[solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]]) for j in range(N_DAYS)]
    
    #每个家庭在10个choice中出现的总数
    family_presense=[solver.Sum(x[i,j] for j in DESIERD[i,:]) for i in families]
    
    #定义目标函数 preference cost部分
    preference_cost=solver.Sum([pcost_mat[i,j] * x[i,j] for i in families for j in DESIERD[i,:]] )
    
    #满足preference_cost最小
    solver.Minimize(preference_cost)
        
    #每个家庭都在10个chioce中出现1次
    for i in range(n_families):
        solver.Add(family_presense[i]==1)
        
    #每天访问人数约束
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j]>=min_occupancy[j])
        solver.Add(daily_occupancy[j]<=min_occupancy[j])
    result=solver.Solve()
    
    temp=[(i,j) for i in families  for j in DESIERD[i,:] if x[i,j].solution_value()>0]
   
    #计算剩余家庭的安排
    df=pd.DataFrame(temp,columns=['family_id','day'])
    return df

In [50]:
#没有安排的familyId 统计出来
unassigned=unassigned_df.family_id.unique()
print(unassigned)
result=solveIP(unassigned,min_occupancy,max_occupancy)
result

[ 188  262  270  612 1192 1557 1728 1770 2011 3181 3534 4064 4193 4223
 4410 4567 4628 4695 4775 4923]


,family_id,day


In [51]:
df=pd.concat((assigned_df[['family_id','day']],result)).sort_values('family_id')
df

,family_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
5074,4995,15
5075,4996,87
5076,4997,31
5077,4998,91


In [55]:
def pcost(prediction):
    daily_occupancy=np.zeros(N_DAYS+1,dtype=np.int64)
    penalty=0
    for (i,p) in enumerate(prediction):
        #计算家庭人数
        n=FAMILY_SIZE[i]
        #第i个家庭，p天访问时的cost
        penalty+=pcost_mat[i,p]
        #计算当头的人数
        daily_occupancy[p]+=n
    return penalty,daily_occupancy

In [56]:
def acost(daily_occupancy):
    accounting_cost=0
    num_out_of_range=0
    daily_occupancy[-1]=daily_occupancy[-2]
    for day in range(N_DAYS):
        n_pl=daily_occupancy[day+1]
        n=daily_occupancy[day]
        #如果超过了承载范围，则设置out_of _range
        num_out_of_range+=(n>MAX_OCCUPANCY) or (n<MIN_OCCUPANCY)
        #计算accouting cost
        accounting_cost += acost_mat[n,n_pl]
    return accounting_cost,num_out_of_range

In [57]:
def cost_function(prediction):
    #基于predicion
    penalty,daily_occupancy=pcost(prediction)#统计preference_cost和每天承载数量
    accounting_cost,num_out_of_range=acost(daily_occupancy)#根据每天承载数量，计算accounting cost
    final_score=penalty+accounting_cost+num_out_of_range*99999999
    return final_score    

In [58]:
prediction=df.day.values
print(cost_function(prediction))

640103479744.4591


In [53]:
def save_result(pred,filename):
    result=pd.DataFrame(range(N_FAMILY),columns=['family_id'])    
    print(result.shape)
    new_result=result.iloc[0:4940,:]
    
    new_result['assigned_day']=pred+1 # 维数相差1，需要+1
    new_result.to_csv(filename,index=False)
    return result
result=save_result(prediction,'submission.csv')

(5000, 1)


In [ ]:
# 其他更好的方案优化还没有做到